To install kerchunk, used
```
!pip install git+https://github.com/fsspec/kerchunk

```

In [1]:
import fsspec
import kerchunk
from kerchunk.hdf import SingleHdf5ToZarr
import ujson
import xarray as xr

In [2]:
from fsspec.registry import known_implementations
known_implementations.keys()

dict_keys(['abfs', 'adl', 'arrow_hdfs', 'asynclocal', 'az', 'blockcache', 'box', 'cached', 'dask', 'data', 'dbfs', 'dir', 'dropbox', 'dvc', 'file', 'filecache', 'ftp', 'gcs', 'gdrive', 'generic', 'git', 'github', 'gs', 'hdfs', 'hf', 'http', 'https', 'jlab', 'jupyter', 'lakefs', 'libarchive', 'local', 'memory', 'oci', 'ocilake', 'oss', 'reference', 'root', 's3', 's3a', 'sftp', 'simplecache', 'smb', 'ssh', 'tar', 'wandb', 'webdav', 'webhdfs', 'zip'])

### Locate S3 endpoints

In [3]:
fs = fsspec.filesystem('s3', anon=True)

In [4]:
flist = fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")
flist

['noaa-goes16/ABI-L2-SSTF/2020/210/00/OR_ABI-L2-SSTF-M6_G16_s20202100000205_e20202100059513_c20202100105456.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/01/OR_ABI-L2-SSTF-M6_G16_s20202100100205_e20202100159512_c20202100205423.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/02/OR_ABI-L2-SSTF-M6_G16_s20202100200204_e20202100259512_c20202100305437.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/03/OR_ABI-L2-SSTF-M6_G16_s20202100300204_e20202100359512_c20202100405457.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/04/OR_ABI-L2-SSTF-M6_G16_s20202100400204_e20202100459511_c20202100505540.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/05/OR_ABI-L2-SSTF-M6_G16_s20202100500203_e20202100559513_c20202100605464.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/06/OR_ABI-L2-SSTF-M6_G16_s20202100600205_e20202100659513_c20202100705486.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/07/OR_ABI-L2-SSTF-M6_G16_s20202100700205_e20202100759512_c20202100805387.nc',
 'noaa-goes16/ABI-L2-SSTF/2020/210/08/OR_ABI-L2-SSTF-M6_G16_s20202100800204_e20202100859512_c202

### Create single kerchunk file

In [5]:
flist = ['s3://' + f for f in flist]
u = flist[0]
u

's3://noaa-goes16/ABI-L2-SSTF/2020/210/00/OR_ABI-L2-SSTF-M6_G16_s20202100000205_e20202100059513_c20202100105456.nc'

In [6]:
%%time
with fsspec.open(u, mode="rb", anon=True) as infile:
    reference = SingleHdf5ToZarr(infile, u, inline_threshold=100).translate()

CPU times: user 2.03 s, sys: 624 ms, total: 2.65 s
Wall time: 38 s


In [7]:
type(reference)

dict

In [8]:
!mkdir './example_jsons/'
!mkdir './example_jsons/individual/'

In [9]:
name = './example_jsons/individual/'+ u.split('/')[-1].replace('.nc', '.json')

with open(name, 'w') as outf:
    outf.write(ujson.dumps(reference))

### Open dataset using kerchunk file

In [12]:
%%time

ds = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": reference,
            "remote_protocol": "s3",
            "remote_options": {"anon": True}
        },
        "consolidated": False
    }
)

CPU times: user 45.3 ms, sys: 7.78 ms, total: 53.1 ms
Wall time: 679 ms


In [11]:
ds

<xarray.Dataset> Size: 353MB
Dimensions:                                                 (y: 5424, x: 5424,
                                                             SST_day_night_emissive_bands: 4,
                                                             SST_night_only_emissive_band: 1,
                                                             number_of_SZA_bounds: 2,
                                                             number_of_LZA_bounds: 2,
                                                             number_of_time_bounds: 2,
                                                             number_of_image_bounds: 2)
Coordinates: (12/14)
    SST_day_night_emissive_band_ids                         (SST_day_night_emissive_bands) float32 16B ...
    SST_day_night_emissive_wavelengths                      (SST_day_night_emissive_bands) float32 16B ...
    SST_night_only_emissive_band_id                         (SST_night_only_emissive_band) int8 1B ...
    SST_night_only_emissive_wavelength                      (SST_night_only_emissive_band) float32 4B ...
    day_solar_zenith_angle                                  float32 4B ...
    night_solar_zenith_angle                                float32 4B ...
    ...                                                      ...
    retrieval_solar_zenith_angle                            float32 4B ...
    t                                                       datetime64[ns] 8B ...
  * x                                                       (x) float64 43kB ...
    x_image                                                 float32 4B ...
  * y                                                       (y) float64 43kB ...
    y_image                                                 float32 4B ...
Dimensions without coordinates: SST_day_night_emissive_bands,
                                SST_night_only_emissive_band,
                                number_of_SZA_bounds, number_of_LZA_bounds,
                                number_of_time_bounds, number_of_image_bounds
Data variables: (12/42)
    DQF                                                     (y, x) float32 118MB ...
    SST                                                     (y, x) float64 235MB ...
    algorithm_dynamic_input_data_container                  float64 8B ...
    algorithm_product_version_container                     float64 8B ...
    day_solar_zenith_angle_bounds                           (number_of_SZA_bounds) float32 8B ...
    geospatial_lat_lon_extent                               float32 4B ...
    ...                                                      ...
    total_number_of_severely_degraded_quality_ocean_pixels  float64 8B ...
    total_number_of_unprocessed_pixels                      float64 8B ...
    twilight_solar_zenith_angle                             float32 4B ...
    twilight_solar_zenith_angle_bounds                      (number_of_SZA_bounds) float32 8B ...
    x_image_bounds                                          (number_of_image_bounds) float32 8B ...
    y_image_bounds                                          (number_of_image_bounds) float32 8B ...
Attributes: (12/33)
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Image
    cell_methods:              quantitative_local_zenith_angle: sum retrieval...
    dataset_name:              OR_ABI-L2-SSTF-M6_G16_s20202100000205_e2020210...
    date_created:              2020-07-28T01:05:45.6Z
    ...                        ...
    summary:                   The ABI Sea Surface Temperature (SST) is calcu...
    time_coverage_end:         2020-07-28T00:59:51.3Z
    time_coverage_start:       2020-07-28T00:00:20.5Z
    timeline_id:               ABI Mode 6
    title:                     ABI L2 Sea Surface (Skin) Temperature
    units:                     K